Oct 15 - work in progress

Instructions
Your task is to implement a simplified version of an in-memory database. All operations that should be supported by this database are described below.

Solving this task consists of several levels. Subsequent levels are opened when the current level is correctly solved. You always have access to the data for the current and all previous levels.

You can execute a single test case by running the following command in the terminal: bash run_single_test.sh "<test_case_name>".

Requirements
Your task is to implement a simplified version of an in-memory database. Plan your design according to the level specifications below:

Level 1: In-memory database should support basic operations to manipulate records, fields, and values within fields.
Level 2: In-memory database should support retrieving statistics about the frequency of updates to records.
Level 3: In-memory database should support multiple users and allow users to lock records.
Level 4: In-memory database should support undo and logout operations from users.
To move to the next level, you need to pass all the tests at this level.

Level 3
Introduce operations to support users locking access to records.

Users can request to temporarily lock a particular record, acquiring exclusive modification access.
If a record is already locked by another user, locking requests are put in a first-in-first-out queue.
Repeated locking requests for a record from the same user should be ignored.
Each modification operation from the previous levels now has an alternative version with a caller_id parameter, which represents a unique string identifier associated with a user. These operations should function the same way as the original version if the user with the given caller_id has locked the record and thus is eligible to modify the record; otherwise, the operation will not change the database. The prior versions should not modify locked records due to the lack of a caller_id to establish permissions. When a user waits for modification access to a locked record, any modification operations they execute on the record are ignored (instead of delayed).

Note that locking should not affect the get operation - values of records can be read at any time, regardless of who has locked access to the record.



`set_or_inc_by_caller(self, key: str, field: str, value: int, caller_id: str) -> int | None` — if a record with the given key does not exist, or is not locked, or the user associated with caller_id is the one who has locked it, perform the set_or_inc operation described in Level 1. Otherwise, ignore the operation and return the existing value if the field exists; otherwise, return None.<br>

`delete_by_caller(self, key: str, field: str, caller_id: str) -> bool` — if the record with the given key exists, and it is either not locked or the user associated with caller_id is the one who has locked it, perform the delete operation described in Level 1. Otherwise, ignore the operation and return False. If the record was deleted (i.e., all the fields of the record associated with key were deleted), all the information related to this record is completely removed from the database, including lock requests.<br>

`lock(self, caller_id: str, key: str) -> str | None` — should request to lock the record associated with key to the user associated with caller_id. After locking a record, the set_or_inc and delete operations cannot be performed on that record, so they should be ignored if called. The operation returns one of the following to signal lock status:<br>

`"acquired"` - if the key is valid and the record is successfully locked to the user; <br>
`"wait"` - if the record is already locked by another user, the user should be added to the queue for locking this record, and will be able to lock the record when it gets released from the previous user;<br>
`None` - if there is an existing lock request from the same user or the record is already locked by the same user;<br>
`"invalid_request"` - if the key doesn't exist in the database. <br>
`unlock(self, key: str) -> str | None` — should release the current lock on the record associated with key. The next user in the lock queue for this record, if any, should automatically obtain a lock. The only way to release the lock from any record is to call the unlock operation explicitly. It does nothing if the record is not currently locked by any users. If the record is not present in the database when unlocked, meaning it was entirely deleted and then unlocked, all lock requests for this record should be deleted. Note that if the key was entirely deleted and re-created with the same key during one lock, it is considered the same record. Returns one of the following to signal lock status:<br>

`"released"` - if the lock was released during the operation, including the case when the record was locked, entirely deleted and then unlocked;
None - if the key exists, but was not locked;<br>
`"invalid_request"` - if the key doesn't exist in the database.<br>

In [ ]:
from in_memory_db import InMemoryDB
from collections import defaultdict, deque

class InMemoryDBImpl(InMemoryDB):
    def __init__(self):
        self.dic = defaultdict(dict)
        self.modification_counts = defaultdict(int)
        self.locks = {}  # To store current locks: {key: caller_id}
        self.lock_queues = defaultdict(deque)  # Queue for lock requests: {key: deque([caller_id1, caller_id2, ...])}

    def set_or_inc(self, key: str, field: str, value: int) -> int:
        # NEP Ignore the caller if locked & userID does not match


        # Instruction of this code reads super confusing.
        if key in self.locks:  # NEP needs to add one more condition here
            return self.dic[key].get(field, 0)
        self.count_call(key)
        if field in self.dic[key]:
            self.dic[key][field] += value
        else:
            self.dic[key][field] = value
        return self.dic[key][field]
        # Need modification

    def get(self, key: str, field: str) -> int | None:
        return self.dic[key].get(field)

    def delete(self, key: str, field: str) -> bool:
        # NEP Ignore the caller if locked & userID does not match
        if key in self.locks:   # NEP needs to add one more condition here
            return False
        if field in self.dic[key]:
            del self.dic[key][field]
            self.count_call(key)
            if not self.dic[key]:
                del self.dic[key]
                del self.modification_counts[key]
            return True
        return False
        # Need modification

    def count_call(self, key: str):
        self.modification_counts[key] += 1

    def top_n_keys(self, n: int) -> list[str]:
        ranked_dict = sorted(self.modification_counts.items(), key=lambda x: (-x[1], x[0]))
        top_n = ranked_dict[:n]
        result = [f"{key}({value})" for key, value in top_n]
        return result


    def set_or_inc_by_caller(self, key: str, field: str, value: int, caller_id: str) -> int | None

    '''
    if a record with the given key does not exist, or is not locked, or the user associated with
    caller_id is the one who has locked it, perform the set_or_inc operation described in Level 1.
    Otherwise, ignore the operation and return the existing value if the field exists;
    otherwise, return None.
    '''
      if key not in self.dic or <record not locked> or caller_id in <allowed_users>:
        set_or_inc(key,field,valie,user_id)

    def delete_by_caller(self, key: str, field: str, caller_id: str) -> bool

    def lock(self, caller_id: str, key: str) -> str | None
      '''
      This func should request to lock the record associated with key to the user associated with caller_id.
      After locking a record, the set_or_inc and delete operations cannot be performed on that record,
      so they should be ignored if called. The operation returns one of the following to signal lock status:
      "acquired" - if the key is valid and the record is successfully locked to the user;
      "wait" - if the record is already locked by another user, the user should be added to the queue for locking this record,
      and will be able to lock the record when it gets released from the previous user;
      None - if there is an existing lock request from the same user or the record is already locked by the same user;
      "invalid_request" - if the key doesn't exist in the database.
      '''
      'lock the record under request on certain conditions (not locked by others)'
      if key in self.dic and key not in self.locks:  # valid key and not already locked
        self.locks[key] = caller_id # locks record with user
        return "acquired"
      if key in self.dic and key in self.locks:  # NEP this should connect to the self.locks in the self.unlock func
        self.lock_queues.append(caller_id)  # add caller_id to the end of th queue
        return "wait"
      if key in self.dic and key in self.locks and caller_id == self.locks[key]: # repetitive request; ignored
        return None
      if key not in key.dic:
        return "invalid_request"

    def unlock(self, key: str) -> str | None
      '''
      1. should release the current lock on the record associated with key.
      2. The next user in the lock queue for this record, if any, should automatically obtain a lock.
      Note: The only way to release the lock from any record is to call the unlock operation explicitly.
      3. It does nothing if the record is not currently locked by any users.
      4. If the record is not present in the database when unlocked, meaning it was entirely deleted and then unlocked,
         all lock requests for this record should be deleted.
      Note: if the key was entirely deleted and re-created with the same key during one lock,
            it is considered the same record. Returns one of the following to signal lock status:

              - "released" - if the lock was released during the operation, including the case when the record was locked, entirely deleted and then unlocked; None - if the key exists, but was not locked;
              - "invalid_request" - if the key doesn't exist in the database.
      '''
      # no record
      if not self.locks[key]:
        return None # Nep: no operation

      # release the lock
      if len(self.lock_queues[key])==0:
        del self.locks[key]  # remove this record from locks if no one in the queue
      else:
        self.locks[key] = self.lock_queues[key].pop() # pop the first user in the key to the locks # NEP to be confirmed
      if not self.dic[key]:
        del self.queues[key]

      # Oct 15 Not finished

















In [ ]:
'''
def set_or_inc_by_caller(self, key: str, field: str, value: int, caller_id: str) -> int | None:
        if key not in self.locks or self.locks[key] == caller_id:
            return self.set_or_inc(key, field, value)
        return self.get(key, field)

    def delete_by_caller(self, key: str, field: str, caller_id: str) -> bool:
        if key not in self.locks or self.locks[key] == caller_id:
            result = self.delete(key, field)
            if key not in self.dic:
                self.remove_all_locks(key)
            return result
        return False

    def lock(self, caller_id: str, key: str) -> str | None:
        if key not in self.dic:
            return "invalid_request"
        if key in self.locks:
            if self.locks[key] == caller_id:
                return None
            if caller_id in self.lock_queues[key]:
                return None
            self.lock_queues[key].append(caller_id)
            return "wait"
        self.locks[key] = caller_id
        return "acquired"

    def unlock(self, key: str) -> str | None:
        if key not in self.dic and key not in self.locks:
            return "invalid_request"
        if key not in self.locks:
            return None
        del self.locks[key]
        if self.lock_queues[key]:
            next_caller = self.lock_queues[key].popleft()
            self.locks[key] = next_caller
        return "released"

    def remove_all_locks(self, key: str):
        if key in self.locks:
            del self.locks[key]
        if key in self.lock_queues:
            del self.lock_queues[key]


'''